# Kick off the Robot Revolution

by learning how to classify the handwritten digits found in the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset!

Although you may want to turn your nose at this quite-solved problem, it is still highly relevant in testing new approaches due to its small size (which means that the models are fast to train!).

In Torch, MNIST is conveniently available from the `mnist` package which we will import using

In [2]:
mnist = require 'mnist'
print(mnist)

{
  testdataset : function: 0x02ed1228
  traindataset : function: 0x02ed11b0
}


The functions that were imported can be called to yield the images and labels:

In [3]:
trainData = mnist.traindataset()
print(trainData)

{
  data : ByteTensor - size: 60000x28x28
  size : 60000
  label : ByteTensor - size: 60000
}


which look like

In [4]:
images = {}
labels = {}
for i=1,5 do
    r = torch.random(trainData.size)
    images[i] = trainData.data[r]
    labels[i] = trainData.label[r]
end
itorch.image(images)
for i=1,5 do io.write(labels[i]..' ') end

4 3 3 5 5 

Seems reasonable. Let's see if our computers agree! 

## DataLoader? I hardly know 'er!

For the remainder of this workshop, you'll be getting your hands dirty by first filling in a very basic skeleton of the data loader and then writing and tuning several models to get the best performance on MNIST!

### One-sample, two-sample, $n$-sample, batch!

Data's come and I want to go learn...

A neural network is like any other function: it takes some inputs (data) and produces some outputs (predictions). Whether you're a mathemetician and think of them as domain and codomain or a software engineer and think of them as arguments and return values, everything starts with getting some data into your network!

In this section, we'll work with the code found in [`mnist/dataloder.lua`](../edit/mnist/dataloader.lua).

Let's start with a quick tour:

The first new piece of machinery is on line 4: 

```lua
local DataLoader = torch.class('DataLoader')
```

If you hearken back to the Intro to Lua segment, you'll recall that Lua tables can be used like classes in standard object-oriented programming. Torch makes this easy by handling all of the boilerplate in [torch.class](https://github.com/torch/torch7/blob/master/doc/utility.md#torch.class) so that all you need to do is implement the methods you care about. In this case, we will want

* `__init` (a default method) which is analagous to `__init__` in Python,
* `run` (a custom method) which will return an iterator over our dataset, and
* `size` (a custom method) which will return the size of the training or validation set

`__init` has been implemented for you. In general, this is where you want to pick apart and reconstruct the arguments (like storing `batchSize` on line 8) and do any one-time data pre-processing like adding one to all of the labels (line 9). This function does not return anything since its job is to build a new instance of the class.

#### Sizing up the problem

As a quick warm-up, let's add a `size` method to the `DataLoader` class that takes the name of a partition (`'train'` or `'val'`) and returns the number of samples (image-label pairs) it contains.

Hint: draw inspiration from the header of the `run` method and scroll up [on this page] to see what's in `self.data`.

When you're done, run the following code block to see if you've implemented the function correctly. If you run into any trouble, just raise your hand and ask!

In [5]:
dofile('mnist/test/dataloader_size.lua')

mnist/test/dataloader_size.lua:7: attempt to call method 'size' (a nil value)
stack traceback:
	mnist/test/dataloader_size.lua:7: in main chunk
	[C]: in function 'f'
	[string "local f = function() return dofile('mnist/tes..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/nhynes/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0102963d10: 

#### I'm gonna run epochs around you!

Now for the real vegetables (not meat because I'm a vegan) of the problem: the `run` method. This one is a bit tricky so we'll work through it together. The first step is to set up the state of the iterator that we will return; this will be used to keep track of what data we still need to send.

Start by assigning to `data` (line 23) the table containing the data associated with the selected `partition`.

Once you've done that, turn your attention to `perm` (line 24). This variable will hold a `LongTensor` containing a random permutation of the numbers `[1..partitionSize]` and will be used to shuffle the data seen during this round of training, or *epoch*. You should start by creating a new [`LongTensor`](https://torch7.readthedocs.io/en/latest/tensor/index.html#tensor-constructors) and then making the appropriate call to [`randperm`](https://torch7.readthedocs.io/en/latest/maths/index.html#res-torchrandpermres-gen-n).

Wrap up the preamble of `run` by making unpacking the `batchSize` into a local variable.

Before continuing on to the iterator, itself, take a gander at the lines

```lua
local permImages = data.data:index(1, perm)
local permLabels = data.label:index(1, perm)
```

If you reference the docs for [`index`](https://torch7.readthedocs.io/en/latest/tensor/index.html#tensor-indexdim-index), you'll notice that these are what do the actual shuffling. That is to say that `permImages` and `permLabels` contain the data, in order, that we'll be feeding into the net (as mini-batches, of course).  
<span style="color: #777; font-size: 90%">Note: index creates a new Storage, so when your data are large, you should, instead, use a minibatch-sized slice of the permutation Tensor to index the data Tensor.</span>

Now then, where were we? Ah, of course: the data iterator! This (`loadData`) is basically a function that gets called repeatedly in a `for` loop and returns non-`nil` values until it runs out of data. It's very similar to a generator in Python. Your task, here, is to use the [`sub`](https://torch7.readthedocs.io/en/latest/tensor/index.html#tensor-subdim1s-dim1e-dim4s-dim4e), [`narrow`](https://torch7.readthedocs.io/en/latest/tensor/index.html#self-narrowdim-index-size), or [\[\] operator](torch_basics.ipynb) to select the correct slices from `permImages` and `permLabels`.

When you're done, you can run the following code block to see if it works! As ever, if you run into trouble, just raise your and and ask :)

In [6]:
dofile('mnist/test/dataloader_run.lua')

/Users/nhynes/dlt7/mnist/dataloader.lua:29: attempt to index local 'data' (a nil value)
stack traceback:
	/Users/nhynes/dlt7/mnist/dataloader.lua:29: in function 'run'
	mnist/test/dataloader_run.lua:16: in main chunk
	[C]: in function 'f'
	[string "local f = function() return dofile('mnist/tes..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/nhynes/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rs/nhynes/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/nhynes/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0102963d10: 

Everything check out okay? Right on! Now you, like Torch, are on fire!

## `nn`: For (Almost) All of Your Neural Network Needs

Another of the many reasons why Torch really shines (glows?) is that it has a comprehensive set of "modules" that you can string together like lego blocks to produce even highly complicated networks. This is largely in part due to the programming abstractions it uses, which we shall soon see.

Like `mnist` we can import the `nn` package using

In [7]:
require 'nn';

Note that this time around, I'm not going to print out the contents of the package since there are [lots and lots](https://github.com/torch/nn#neural-network-package) of modules! Most people think that Torch has a steep learning curve for this reason. Rest assured, though, that by browsing the list of modules enough times, you too will become highly proficinnent.

### The Real Learning Modules

All of the modules in `nn` are built on a base class known as [`nn.Module`](https://github.com/torch/nn/blob/master/Module.lua). Thus, in order to understand the rest of the package, we must first understand [`nn.Module`](https://github.com/torch/nn/blob/master/Module.lua). The [docs](https://github.com/torch/nn/blob/master/doc/module.md) are comprehensive but somewhat hard to parse, so we'll eschew those and just look at the code:

You'll want to point your browser to the code for [`nn.Module`](https://github.com/torch/nn/blob/master/Module.lua).

Now, direct your attention to the [`init`](https://github.com/torch/nn/blob/master/Module.lua#L3) method. The two important things going on here are the initialization of `self.gradInput` and `self.output`. The former holds the gradient that travels backwards through this layer (think [$\partial f/\partial z$](neural_nets.ipynb#Training)) while the latter holds the output. Practically, this means is that you can access these important values using `myModule.gradInput` and `myModule.output`, respectively. There are some other effects that have to deal with the `_type`, but we'll ignore that for now.

Temporarily skipping over the `parameters` method, we eventually come to [`forward`](https://github.com/torch/nn/blob/master/Module.lua#L25), which just calls `updateOutput`; you generally don't want to change this and, instead, override [`updateOutput`](https://github.com/torch/nn/blob/master/Module.lua#L21).

[`updateOutput`](https://github.com/torch/nn/blob/master/Module.lua#L21) is where (half of) the real work of the Module happens. You override (replace) this function with whatever computation you desire and then make `self.output` the result.

Continuing on, we arrive at [`backward`](https://github.com/torch/nn/blob/master/Module.lua#L29), which is a bit like `forward` in that it simply calls some other methods: `updateGradInput` and `accGradParameters`.

Try your hardest to ignore all of the intervening code and settle your gaze upon [`updateGradInput`](https://github.com/torch/nn/blob/master/Module.lua#L42). This function is the backward counterpart of `updateOutput` which takes the gradient w.r.t. the output (e.g., $\frac{\partial \mathcal{L}}{\partial o}$) and applies the chain rule (e.g., $\frac{\partial \mathcal{L}}{\partial o}\frac{\partial o}{\partial f}$) to get the gradient w.r.t. the input--the output of the previous module. Naturally, this gets stored in `self.gradInput`.

"What about the parameters?" you say. Great question, that's what happens in [`accGradParameters`](https://github.com/torch/nn/blob/master/Module.lua#L46). This function doesn't return anything and, instead, just collects the $\partial z/\partial W$s until we take a gradient descent step at the very end.

Clear as mud, right? No worries, it should become clearer when we get to an example!

#### Mulling Over a Module

Let's scale up the learning with a simple Module, [`nn.Mul`](https://github.com/torch/nn/blob/master/Mul.lua), which learns to multiply the input by a number, $c$. In math parlance, $f(X) = cX$.

On [line 1](https://github.com/torch/nn/blob/master/Mul.lua#L1), you can see that we're creating `nn.Mul` from a base class `nn.Module` (go figure).

In [`__init`](https://github.com/torch/nn/blob/master/Mul.lua#L4) the new thing happening here is that there are two new variables: `self.weight` and `self.gradWeight` which hold the number and the accumulated gradient w.r.t. the number ($\partial\mathcal{L}/dc$). Since this is a `nn.Module` and it calls [`parent.__init(self)`](https://github.com/torch/nn/blob/master/Mul.lua#L4), we know that `self.output` and `self.gradInput` are initialized to empty Tensors.

[`updateOutput`](https://github.com/torch/nn/blob/master/Mul.lua#L23) is reasonably straightforward: it makes some room for the output using `resize` and copies in the input (it's important to do a copy, because it needs the input later to compute the gradient!). After a quick `mul` by $c$, the module returns its output.

[`updateGradInput`](https://github.com/torch/nn/blob/master/Mul.lua#L29) proceeds similarly. This function computes $\partial f/\partial x = c$, and applies the chain rule so that `gradInput = c*gradOutput`. Don't let the call to [`add`](https://torch7.readthedocs.io/en/latest/maths/index.html#res-torchaddres-tensor1-value-tensor2) throw you off: if you check the types (scalar, Tensor) and then look at the [docs](https://torch7.readthedocs.io/en/latest/maths/index.html#res-torchaddres-tensor1-value-tensor2) (scroll down to the first example in red), you'll find that this invocation

> `x:add(value, y)` multiply-accumulates values of `y` into `x`"

Great! Now on to the final piece of the puzzle, [`accGradParameters`](https://github.com/torch/nn/blob/master/Mul.lua#L35), which just takes $\partial f/\partial c = X$, applies the chain rule, and adds the result into the existing `gradWeight`.

In [8]:
mul42 = nn.Mul()
mul42.weight = torch.Tensor{42}
print(mul42)

nn.Mul
{
  weight : DoubleTensor - size: 1
  _type : torch.DoubleTensor
  output : DoubleTensor - empty
  gradInput : DoubleTensor - empty
  gradWeight : DoubleTensor - size: 1
}


In [9]:
input = torch.range(1, 3)
mul42:forward(input)
print(mul42.output)

  42
  84
 126
[torch.DoubleTensor of size 3]



In [10]:
gradOutput = torch.ones(3)
mul42:backward(input, gradOutput)
print(mul42.gradInput)
print(mul42.gradWeight) -- 1 + 2 + 3 = input:sum()

 42
 42
 42
[torch.DoubleTensor of size 3]

 6
[torch.DoubleTensor of size 1]



### So Many Modules... I Can Hardly Contain My Excitement!

Whereas a vanilla Module is great when you have an input and want to create a new output, most often, you'll want to string other Modules together. For this, you need a [`Container`](https://github.com/torch/nn/blob/master/Container.lua). A [`Container`](https://github.com/torch/nn/blob/master/Container.lua) extends Module to provide, in addition to our friend like `self.output`, `self.gradInput`, `updateOutput()`, and `updateGradInput()`, the `self.modules` property and methods like [`add`](https://github.com/torch/nn/blob/master/Container.lua#L10) and the recursive [`applyToModules`](https://github.com/torch/nn/blob/master/Container.lua#L10).

By far, the most important part of being a Container is that its contained Modules are in the `self.modules` table. Otherwise, the rest of the Container functions won't know to use them!

#### It's as Easy as 1, 2, 3

By far, the most ubiquitous Module is [`nn.Sequential`](https://github.com/torch/nn/blob/master/Sequential.lua), which [takes an input and feeds it--sequentially--through the network](https://github.com/torch/nn/blob/master/Sequential.lua#L41). As this, very much, follows the paradigm of "layers," you'll not be surprised that the wrapper of most of your networks will be `nn.Sequential`.

Here's a quick example that looks a lot like our Perceptron from earlier:

In [22]:
model = nn.Sequential()
    :add(nn.Linear(3, 5))
    :add(nn.ReLU())
    :add(nn.Linear(5, 5))
    :add(nn.ReLU())
    :add(nn.Linear(5, 1))
print(tostring(model)) -- the tostring is only needed in iTorch

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
  (1): nn.Linear(3 -> 5)
  (2): nn.ReLU
  (3): nn.Linear(5 -> 5)
  (4): nn.ReLU
  (5): nn.Linear(5 -> 1)
}	


In [23]:
input = torch.rand(4, 3) -- minibatch of 4 samples with 3 features each
model:forward(input)
for i=1,#model do
    print(model:get(i).output)
end

-0.3423 -0.0190  0.2330 -0.0752  0.1923
 0.0450  0.0384  0.4796  0.1291  0.6151
 0.1226  0.0538  0.2572  0.0938  0.6989
-0.0265  0.0222  0.1911  0.0377  0.5484
[torch.DoubleTensor of size 4x5]

 0.0000  0.0000  0.2330  0.0000  0.1923
 0.0450  0.0384  0.4796  0.1291  0.6151
 0.1226  0.0538  0.2572  0.0938  0.6989
 0.0000  0.0222  0.1911  0.0377  0.5484
[torch.DoubleTensor of size 4x5]



-0.2359  0.2062  0.3793  0.4782  0.4382
-0.0865  0.1853  0.3324  0.5416  0.4416
-0.0157  0.1165  0.3157  0.4999  0.4281
-0.1061  0.1077  0.2999  0.5159  0.4161
[torch.DoubleTensor of size 4x5]

 0.0000  0.2062  0.3793  0.4782  0.4382
 0.0000  0.1853  0.3324  0.5416  0.4416
 0.0000  0.1165  0.3157  0.4999  0.4281
 0.0000  0.1077  0.2999  0.5159  0.4161
[torch.DoubleTensor of size 4x5]

-0.1067
-0.1167
-0.0904
-0.0906
[torch.DoubleTensor of size 4x1]



### The Final Judgement

So, you have a model. Cool. Is it any good? Who knows! Actually, the [`Criterion`](https://github.com/torch/nn/blob/master/Criterion.lua) does.

A [`Criterion`](https://github.com/torch/nn/blob/master/Criterion.lua) is kind of like a Module in that it has  `forward` and `backward`, but is unique in that they take a second argument: `target`.

As with Modules, there are a [ton](https://github.com/torch/nn/blob/master/doc/criterion.md) of different Criterions. Choosing the right Criterion is very important as this determines what the network will learn.

### All together now

So now we have a network that takes a vector with three elements and produces a single number. How about we train it to learn how to take the average of the three numbers?

The question of the hour is of what Criterion should we use. One good choice is [`nn.MSECriterion`](https://github.com/torch/nn/blob/master/doc/criterion.md#nn.MSECriterion) which is the $\mathcal{L}(x, y) = (x - y)^2$ we used earlier. This is sort of like asking the output to be "close enough" while ensuring that no value is particularly off.

Here's how the network performs before any training:

In [26]:
origModel = model:clone()
origModel:reset()

testData = torch.range(1, 4*3):view(4, 3)
print(testData)
expectedOutput = testData:mean(2)
print(expectedOutput)

  1   2   3
  4   5   6
  7   8   9
 10  11  12
[torch.DoubleTensor of size 4x3]

  2
  5
  8
 11
[torch.DoubleTensor of size 4x1]



In [27]:
print(origModel:forward(testData))

 0.7178
 0.7694
 0.7868
 0.8480
[torch.DoubleTensor of size 4x1]



Terrible. Absolutely terrible. Let's see if we can get it to learn something!

In [28]:
model:reset()
crit = nn.AbsCriterion() -- (x - y)^2
N = 100000 -- train for 100k iters
for i=1,N do
    local input = torch.Tensor(100, 3):random(10)
    local target = input:max(2)
    model:forward(input)
    local loss = crit:forward(model.output, target)
    
    model:zeroGradParameters()
    crit:backward(model.output, target)
    model:backward(input, crit.gradInput)
    model:updateParameters(0.01)
    
    xlua.progress(i, N)
end

 [=======================================================>]                                Tot: 18s494ms | Step: 0ms 

In [29]:
print(model:forward(testData), expectedOutput)

  3.0216
  5.9010
  8.4828
 11.2717
[torch.DoubleTensor of size 4x1]

  2
  5
  8
 11
[torch.DoubleTensor of size 4x1]



Not too shabby for only being able to do sums, multiplications, and thresholding at zero!

## Go Forth (and Back)

You now have all of the tools you need to start classifying MNIST digits! Crazy, right?

You'll now want to pop open file that will contain your  as well as the [training script](/edit/mnist/main.lua).

### I Want to Be the Very Best...

Start by popping open the [training script](/edit/mnist/main.lua).

You don't have to concern youself much with the details of it but it will help to be aware of a few things:

First is that that there are several named arguments that you can change like `seed`, `nEpochs`, any of the options listed under "Optimization options".

Additionally, on line 39, you'll see that we're using the [`nn.CrossEntropyCriteron`](https://github.com/torch/nn/blob/master/doc/criterion.md#nn.CrossEntropyCriterion) which is like the binary cross-entropy we used in the original Perceptron example but generalized to $n$ classes.

Finally, unlike above where we used `updateParamters(0.01)`, this training script uses the SGD available in the powerful `optim` package which has, along with other optimizers entirely, an implementation of SGD that includes bells and whistles like weight-decay and momentum. This is configured on line 41 and is used on line 75.

### I'm Ready for My Close-Up

Finally, we get to write a real-life model! This model will live in [mnist/models/mlp.lua](/edit/mnist/models/mlp.lua) which is named "mlp.lua" because that's what you're going to write: a multi-layer perceptron!

In the spirit of the max-finding model, above, you should--at the very least--put together a model that contains some [`nn.Linear`s](https://github.com/torch/nn/blob/master/doc/simple.md#nn.Linear) and some [non-linearities](https://github.com/torch/nn/blob/master/doc/transfer.md) like [`nn.ReLU`](https://github.com/torch/nn/blob/master/doc/transfer.md#relu). You are encouraged to experiment with other modules!

**Important:** remember to keep tabs on your input and output dimensions! The first thing that you should notice is that the input is a $N\times 28 \times 28$ Tensor (where $N$ is the batch size). This is a problem if you want to feed it into a `nn.Linear` since `nn.Linear` only takes $N \times d$ Tensors. You will need to flatten your image first using [`nn.View`](https://github.com/torch/nn/blob/master/doc/simple.md#view) (scroll down to "Example 2" for an... example).

You can test that your model conforms to the spec by running the code below:

In [45]:
dofile('mnist/test/mlp_io.lua')

When you're ready to roll, run the following code block. Try to get the best performance you can!

In [ ]:
trainMNIST = dofile('mnist/main.lua')

trainMNIST()
-- trainMNIST({dispFreq=2, nEpochs=10, batchSize=1024, lr=0.01}) -- twiddle some hyperparameters

# Congratulations!

What did you get? 94%, 96%? That's it?? Don't talk to me until you're at least at 99.7%

"But... but.. I used over 9000 linear layers!" you say in self-righteous indignation. Well sure you did and therein lies part of the problem! A simple MLP, on its own, ignores the 2D structure present within the input signal. To take advantage of that, we'll need a new architecture... Up next: convnets!